## Combine trip files into one single file for duaiterate.py to process

### Get path to od files generated separately

In [125]:
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element
from lxml import etree
from copy import copy
import os
import inspect
from xml.dom import minidom

In [126]:
# Path to folder with the sumo files
this_folder = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
data_folder = 'xml_trips'

In [127]:
print(this_folder)

/home/cuip/Documents/CUIP LAPTOP/combine _trips/trips


### Initialize combine trip file

In [128]:
xroutes = Element('routes')
xroutes.set('xmlns:xsi','http://www.w3.org/2001/XMLSchema-instance')
xroutes.set('xsi:noNamespaceSchemaLocation', 'http://sumo.dlr.de/xsd/routes_file.xsd')

In [129]:
# Additing vehicle type definition for truck and trailer
vtype = Element('vType')
vtype.set('id', 'truck')
vtype.set('vClass', 'truck')
xroutes.append(vtype)
vtype1 = Element('vType')
vtype1.set('id', 'trailer')
vtype1.set('vClass', 'trailer')
xroutes.append(vtype1)
vtype1 = Element('vType')
vtype1.set('id', 'passenger')
vtype1.set('vClass', 'passenger')
xroutes.append(vtype1)

### Import od file and combine 

In [130]:
prefix = ['pass_', 'sut_', 'mut_']
#prefix = ['sut']
suffix = ['am_', 'pm_', 'op_']
num_period = 5   #Number of time period

trip_list = []

for p in prefix:
    for s in suffix:
        trip_file = 'trips_' + p  + s + 'xml.xml'
        print(trip_file)
        path_file = os.path.join(this_folder,data_folder, trip_file)
        root_tree = ET.parse(path_file).getroot()
        trip_list.extend(root_tree)

trips_pass_am_xml.xml
trips_pass_pm_xml.xml
trips_pass_op_xml.xml
trips_sut_am_xml.xml
trips_sut_pm_xml.xml
trips_sut_op_xml.xml
trips_mut_am_xml.xml
trips_mut_pm_xml.xml
trips_mut_op_xml.xml


In [131]:
print(trip_list[699498].tag, trip_list[699498].attrib)

trip {'id': 'mut_op454', 'depart': '86397.88', 'from': '-28679', 'to': '-35756', 'type': 'trailer', 'fromTaz': '26', 'toTaz': '719', 'departLane': 'best', 'departPos': 'free', 'departSpeed': 'max'}


In [133]:
# Sort the elements by departure time
trip_list[:] = sorted(trip_list, key=lambda child: (child.tag,float(child.get('depart'))))

In [142]:
print(trip_list[10].tag, trip_list[10].attrib)

trip {'id': 'trip_prefix10', 'depart': '5.38', 'from': '-23263', 'to': '21298', 'type': 'passenger', 'fromTaz': '159', 'toTaz': '32', 'departLane': 'best', 'departPos': 'free', 'departSpeed': 'max'}


In [139]:
for num,s in enumerate(trip_list):
        s.attrib['id'] = 'trip_prefix' + str(num)

In [140]:
# Add trips to xroute
xroutes.extend(trip_list)

In [137]:
file_name = 'combined_trips.xml'
print("Saving to xml: ", file_name)
configfile = os.path.join(this_folder,data_folder, file_name)
with open(configfile, 'wb') as f:
    f.write(minidom.parseString(ET.tostring(xroutes)).toprettyxml(encoding="utf-8"))
    

Saving to xml:  combined_trips.xml
